In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Data Mining Assignment 3: Solutions for Chapter 8 and Chapter 9\n",
    "\n",
    "This Jupyter Notebook provides analytical solutions and Python code for questions from Chapter 8 (Tree-Based Methods) and Chapter 9 (Support Vector Machines) from the provided document. All code has been updated to use `fetch_california_housing` instead of the deprecated `load_boston` dataset to ensure compatibility with modern `scikit-learn` versions.\n",
    "\n",
    "## Chapter 8: Tree-Based Methods"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Question 2: Boosting with Depth-One Trees (Stumps) and Additive Models\n",
    "\n",
    "**Analytical Solution:**\n",
    "\n",
    "The question asks why boosting with depth-one trees (stumps) results in an additive model of the form \( f(X) = \\sum_{j=1}^p f_j(X_j) \). We start with Equation (8.12) from Algorithm 8.2 (Gradient Boosting).\n",
    "\n",
    "In gradient boosting, the model is updated iteratively:\n",
    "\n",
    "\\[ f_m(X) = f_{m-1}(X) + \\alpha_m h_m(X), \\]\n",
    "\n",
    "where \( h_m(X) \) is a stump, and \( \\alpha_m \) is the learning rate. A stump splits on one feature \( X_j \):\n",
    "\n",
    "\\[ h_m(X) = \\begin{cases} \n",
    "c_{m1} & \\text{if } X_j \\leq t_m, \\\\\n",
    "c_{m2} & \\text{if } X_j > t_m, \n",
    "\\end{cases} \\]\n",
    "\n",
    "Thus, \( h_m(X) = h_m(X_j) \). The final model after \( M \) iterations is:\n",
    "\n",
    "\\[ f(X) = \\sum_{m=1}^M \\alpha_m h_m(X_{j_m}). \\]\n",
    "\n",
    "Group terms by feature:\n",
    "\n",
    "\\[ f(X) = \\sum_{j=1}^p \\sum_{m: j_m = j} \\alpha_m h_m(X_j). \\]\n",
    "\n",
    "Define \( f_j(X_j) = \\sum_{m: j_m = j} \\alpha_m h_m(X_j) \), so:\n",
    "\n",
    "\\[ f(X) = \\sum_{j=1}^p f_j(X_j). \\]\n",
    "\n",
    "This is additive because each stump depends on one feature, and boosting combines them linearly. Stumps cannot model feature interactions, ensuring additivity.\n",
    "\n",
    "**Code**: No computation is required for this theoretical question."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Question 7: Random Forests on California Housing Data\n",
    "\n",
    "**Analytical Solution:**\n",
    "\n",
    "We need to plot test error (MSE) for random forests on a housing dataset, varying `max_features` and `n_estimators`, similar to Figure 8.10. The original problem used the Boston dataset (13 features), but due to deprecation, we use `fetch_california_housing` (8 features). We test `max_features = [1, 3, 5, 8]` and `n_estimators = [10, 50, 100, 200, 500]`.\n",
    "\n",
    "Random forests average predictions from multiple trees, reducing variance. We expect:\n",
    "- MSE decreases as `n_estimators` increases, stabilizing around 100–200 trees.\n",
    "- Intermediate `max_features` (e.g., 3, 5) often performs best, balancing randomness and predictive power.\n",
    "- Low `max_features` (e.g., 1) may underfit; high values (e.g., 8) may overfit slightly.\n",
    "\n",
    "**Code**: The following code trains random forests and plots test MSE using the California housing dataset."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%matplotlib inline\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from sklearn.datasets import fetch_california_housing\n",
    "from sklearn.ensemble import RandomForestRegressor\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import mean_squared_error\n",
    "\n",
    "# Load California housing dataset\n",
    "california = fetch_california_housing()\n",
    "X = california.data\n",
    "y = california.target\n",
    "\n",
    "# Split data\n",
    "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n",
    "\n",
    "# Parameters to test\n",
    "n_estimators_list = [10, 50, 100, 200, 500]\n",
    "max_features_list = [1, 3, 5, 8]\n",
    "results = {}\n",
    "\n",
    "# Train models and compute test MSE\n",
    "for max_features in max_features_list:\n",
    "    mse_list = []\n",
    "    for n_estimators in n_estimators_list:\n",
    "        rf = RandomForestRegressor(\n",
    "            n_estimators=n_estimators,\n",
    "            max_features=max_features,\n",
    "            random_state=42\n",
    "        )\n",
    "        rf.fit(X_train, y_train)\n",
    "        y_pred = rf.predict(X_test)\n",
    "        mse = mean_squared_error(y_test, y_pred)\n",
    "        mse_list.append(mse)\n",
    "    results[max_features] = mse_list\n",
    "\n",
    "# Plot results\n",
    "plt.figure(figsize=(10, 6))\n",
    "for max_features in max_features_list:\n",
    "    plt.plot(n_estimators_list, results[max_features], marker='o', label=f'max_features={max_features}')\n",
    "plt.xlabel('Number of Trees (n_estimators)')\n",
    "plt.ylabel('Test MSE')\n",
    "plt.title('Test Error vs. Number of Trees for Different max_features (California Housing)')\n",
    "plt.legend()\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Expected Results**:\n",
    "\n",
    "- **Trend**: MSE decreases with more trees, stabilizing at 100–200 trees.\n",
    "- **max_features**:\n",
    "  - `max_features = 3, 5`: Typically yield lower MSE, balancing randomness and feature inclusion.\n",
    "  - `max_features = 1`: Higher MSE due to underfitting from excessive randomness.\n",
    "  - `max_features = 8`: Slightly higher MSE than 3 or 5, as trees become more correlated.\n",
    "- **Example MSE**: At `n_estimators = 500`, MSE may be ~0.2–0.3, with `max_features = 3` or `5` near the lowest.\n",
    "- The plot resembles Figure 8.10, with intermediate `max_features` typically optimal."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Chapter 9: Support Vector Machines"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Question 1: Hyperplanes in Two Dimensions\n",
    "\n",
    "**Analytical Solution:**\n",
    "\n",
    "We sketch two hyperplanes in 2D and indicate their positive and negative regions.\n",
    "\n",
    "**(a) Hyperplane \( 1 + 3X_1 - X_2 = 0 \)**:\n",
    "\n",
    "\\[ X_2 = 3X_1 + 1. \\]\n",
    "\n",
    "- Line: Slope 3, y-intercept 1. Points: \( (0, 1) \), \( (-\\frac{1}{3}, 0) \).\n",
    "- Regions:\n",
    "  - \( 1 + 3X_1 - X_2 > 0 \): Below the line (\( X_2 < 3X_1 + 1 \)).\n",
    "  - \( 1 + 3X_1 - X_2 < 0 \): Above the line (\( X_2 > 3X_1 + 1 \)).\n",
    "\n",
    "**(b) Hyperplane \( -2 + X_1 + 2X_2 = 0 \)**:\n",
    "\n",
    "\\[ X_2 = -\\frac{1}{2}X_1 + 1. \\]\n",
    "\n",
    "- Line: Slope \(-\\frac{1}{2}\), y-intercept 1. Points: \( (0, 1) \), \( (2, 0) \).\n",
    "- Regions:\n",
    "  - \( -2 + X_1 + 2X_2 > 0 \): Above the line (\( X_2 > -\\frac{1}{2}X_1 + 1 \)).\n",
    "  - \( -2 + X_1 + 2X_2 < 0 \): Below the line (\( X_2 < -\\frac{1}{2}X_1 + 1 \)).\n",
    "\n",
    "**Code**: The following code plots both hyperplanes and their regions."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%matplotlib inline\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# Create grid\n",
    "x1 = np.linspace(-5, 5, 100)\n",
    "x2 = np.linspace(-5, 5, 100)\n",
    "X1, X2 = np.meshgrid(x1, x2)\n",
    "\n",
    "# Hyperplane 1: 1 + 3X_1 - X_2 = 0\n",
    "Z1 = 1 + 3*X1 - X2\n",
    "# Hyperplane 2: -2 + X_1 + 2X_2 = 0\n",
    "Z2 = -2 + X1 + 2*X2\n",
    "\n",
    "plt.figure(figsize=(8, 8))\n",
    "\n",
    "# Plot hyperplane 1\n",
    "plt.contour(X1, X2, Z1, levels=[0], colors='blue', linestyles='solid')\n",
    "plt.contourf(X1, X2, Z1, levels=[0, np.max(Z1)], colors='lightblue', alpha=0.3)  # X_2 < 3X_1 + 1\n",
    "plt.contourf(X1, X2, Z1, levels=[np.min(Z1), 0], colors='lightgray', alpha=0.3)  # X_2 > 3X_1 + 1\n",
    "\n",
    "# Plot hyperplane 2\n",
    "plt.contour(X1, X2, Z2, levels=[0], colors='red', linestyles='dashed')\n",
    "plt.contourf(X1, X2, Z2, levels=[0, np.max(Z2)], colors='lightcoral', alpha=0.3)  # X_2 > -X_1/2 + 1\n",
    "plt.contourf(X1, X2, Z2, levels=[np.min(Z1), 0], colors='lightyellow', alpha=0.3)  # X_2 < -X_1/2 + 1\n",
    "\n",
    "plt.xlabel('X_1')\n",
    "plt.ylabel('X_2')\n",
    "plt.title('Hyperplanes and Regions')\n",
    "plt.grid(True)\n",
    "plt.legend(['1 + 3X_1 - X_2 = 0', '-2 + X_1 + 2X_2 = 0'])\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Question 2: Non-Linear Decision Boundary\n",
    "\n",
    "**Analytical Solution:**\n",
    "\n",
    "**(a) Sketch the curve \( (1 + X_1)^2 + (2 - X_2)^2 = 4 \)**:\n",
    "\n",
    "\\[ (X_1 + 1)^2 + (X_2 - 2)^2 = 4. \\]\n",
    "\n",
    "- Circle with center \( (-1, 2) \), radius 2.\n",
    "\n",
    "**(b) Regions**:\n",
    "\n",
    "- \( (1 + X_1)^2 + (2 - X_2)^2 > 4 \): Outside the circle.\n",
    "- \( (1 + X_1)^2 + (2 - X_2)^2 < 4 \): Inside the circle.\n",
    "\n",
    "**(c) Classify points**:\n",
    "\n",
    "- Blue class: \( (1 + X_1)^2 + (2 - X_2)^2 > 4 \).\n",
    "- Red class: \( (1 + X_1)^2 + (2 - X_2)^2 \\leq 4 \).\n",
    "\n",
    "- \( (0, 0) \): \( (1 + 0)^2 + (2 - 0)^2 = 5 > 4 \), Blue.\n",
    "- \( (-1, 1) \): \( (1 - 1)^2 + (2 - 1)^2 = 1 < 4 \), Red.\n",
    "- \( (2, 2) \): \( (1 + 2)^2 + (2 - 2)^2 = 9 > 4 \), Blue.\n",
    "- \( (3, 8) \): \( (1 + 3)^2 + (2 - 8)^2 = 52 > 4 \), Blue.\n",
    "\n",
    "**(d) Linear in transformed space**:\n",
    "\n",
    "Expand: \( X_1^2 + X_2^2 + 2X_1 - 4X_2 + 1 = 0 \).\n",
    "\n",
    "In features \( Z_1 = X_1, Z_2 = X_2, Z_3 = X_1^2, Z_4 = X_2^2 \), the boundary is:\n",
    "\n",
    "\\[ Z_3 + Z_4 + 2Z_1 - 4Z_2 + 1 = 0, \\]\n",
    "\n",
    "which is linear in \( Z_1, Z_2, Z_3, Z_4 \).\n",
    "\n",
    "**Code**: The following code plots the circle and classifies points."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%matplotlib inline\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "# Create grid\n",
    "x1 = np.linspace(-5, 5, 100)\n",
    "x2 = np.linspace(-5, 5, 100)\n",
    "X1, X2 = np.meshgrid(x1, x2)\n",
    "\n",
    "# Circle equation: (1 + X_1)^2 + (2 - X_2)^2\n",
    "Z = (1 + X1)**2 + (2 - X2)**2\n",
    "\n",
    "plt.figure(figsize=(8, 8))\n",
    "\n",
    "# Plot circle\n",
    "plt.contour(X1, X2, Z, levels=[4], colors='black', linestyles='solid')\n",
    "plt.contourf(X1, X2, Z, levels=[4, np.max(Z)], colors='lightblue', alpha=0.3)  # Outside\n",
    "plt.contourf(X1, X2, Z, levels=[np.min(Z), 4], colors='lightcoral', alpha=0.3)  # Inside\n",
    "\n",
    "# Plot points\n",
    "points = [(0, 0), (-1, 1), (2, 2), (3, 8)]\n",
    "colors = []\n",
    "for x1, x2 in points:\n",
    "    value = (1 + x1)**2 + (2 - x2)**2\n",
    "    color = 'blue' if value > 4 else 'red'\n",
    "    colors.append(color)\n",
    "    plt.scatter(x1, x2, c=color, s=100, edgecolors='black')\n",
    "    plt.text(x1 + 0.2, x2, f'({x1}, {x2})', fontsize=10)\n",
    "\n",
    "plt.xlabel('X_1')\n",
    "plt.ylabel('X_2')\n",
    "plt.title('Non-Linear Decision Boundary and Classified Points')\n",
    "plt.grid(True)\n",
    "plt.show()"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined